# UE08: Datenaufbereitung und EDA am Beispiel Titanic

In dieser Übung werden wir den Titanic-Datensatz systematisch analysieren, um erste Erkenntnisse über die Überlebenschancen der Passagiere zu gewinnen. Dabei folgen wir einer bewährten Reihenfolge, um zuverlässige und aussagekräftige Ergebnisse zu erhalten. Zunächst laden wir die Daten und verschaffen uns einen Überblick über deren Struktur und Inhalt. Anschließend bereinigen wir den Datensatz, indem wir fehlende Werte, unplausible Einträge und potenzielle Ausreißer identifizieren und entsprechend behandeln. Erst nach dieser Bereinigung testen wir unsere Hypothesen, indem wir verschiedene Analysen und Visualisierungen durchführen. Diese strukturierte Vorgehensweise stellt sicher, dass unsere Analysen auf einer soliden Datenbasis beruhen und wir fundierte Aussagen treffen können.

## 0. Dataset im Überblick
- **PassengerId**: Eindeutige ID für jeden Passagier
- **Survived**: 1 = Überlebt, 0 = Nicht überlebt
- **Pclass**: Ticketklasse (1 = Erste Klasse, 2 = Zweite Klasse, 3 = Dritte Klasse)
- **Sex**: Geschlecht
- **Age**: Alter des Passagiers
- **SibSp**: Anzahl der Geschwister/Ehepartner an Bord
- **Parch**: Anzahl der Eltern/Kinder an Bord
- **Ticket**: Ticketnummer
- **Fare**: Ticketpreis
- **Cabin**: Kabinennummer
- **Embarked**: Hafen, an dem der Passagier eingestiegen ist (C = Cherbourg, Q = Queenstown, S = Southampton)

## 1. Vorgehensweise
Daten laden und Überblick verschaffen

    - Datensatz einlesen, erste Zeilen anschauen.
    - Infos zu Datentypen und fehlenden Werten prüfen.

Datenbereinigung

    - Fehlende Werte behandeln (z. B. durch Entfernen oder Imputieren).
    - Unplausible Werte und Ausreißer identifizieren.
    - Datentypen anpassen (z. B. Kategorien korrekt kodieren).

Hypothesen testen

    - Explorative Analyse durchführen (Univariate/Bivariate Analyse).
    - Die bereinigten Daten für Visualisierungen und statistische Berechnungen nutzen.
    
## 3. Hypothesen

1. Frauen hatten eine höhere Überlebensrate als Männer.
       
       - Begründung: Historisch gesehen wurden Frauen und Kinder bei der Evakuierung priorisiert („Women and children first“).
       - Überprüfung: Vergleich der Überlebensraten zwischen Männern und Frauen.

2. Passagiere der ersten Klasse hatten eine höhere Überlebenswahrscheinlichkeit als die der dritten Klasse.
        
       - Begründung: Personen in der ersten Klasse hatten besseren Zugang zu Rettungsbooten.
       - Überprüfung: Analyse der Überlebensraten nach Ticketklasse (Pclass).

3. Jüngere Passagiere hatten eine höhere Überlebenschance.

        - Begründung: Kinder wurden bevorzugt gerettet.
        - Überprüfung: Untersuchung der Altersverteilung zwischen Überlebenden und Nicht-Überlebenden.

4. Der Ticketpreis (Fare) könnte mit der Überlebensrate korrelieren.
    
        - Begründung: Höhere Ticketpreise sind mit der ersten Klasse verbunden, die eine höhere Überlebenswahrscheinlichkeit hatte.
        - Überprüfung: Korrelation zwischen Ticketpreis und Überlebensstatus.

5. Passagiere, die alleine reisten, hatten eine geringere Überlebenschance.
    
        - Begründung: Familien könnten sich gegenseitig unterstützt haben.
        - Überprüfung: Zusammenhang zwischen der Anzahl der Familienmitglieder an Bord (SibSp und Parch) und der Überlebensrate.
---
> Für die eine oder andere Aufgabe ist `groupby` überaus hilfreich: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html